In [20]:
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
import sys
from datetime import date, timedelta
import datetime

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [32]:
all_data = pd.read_csv('/home/tiger/archived-data/aeolus-data/20210120/21/36562713-Adhoc-测试-入群-查询22.csv', encoding = 'gb18030')
all_data.head()

,install_date,user_id,channel_group,city_level,age,gender,edu,finish_test,enter_wechat_group
0,2021-01-13,2.374601e+15,OV,新一线,24-30,male,NaN,1,0
1,2021-01-06,1.882018e+15,Google-FB,六线及以下,24-30,female,NaN,1,1
2,2021-01-14,3.729200e+15,华为小米,五线,31-40,female,undergraduate,0,0
3,2021-01-19,1.882778e+15,华为小米,三线,31-40,male,high,0,0
4,2021-01-04,1.380621e+15,Apple Store,新一线,18-23,male,master,0,0


#### 数据预处理
###### 全量视频课体验营

In [33]:
all_data = all_data.fillna('Unknown')
data = all_data[all_data.install_date >= '2021-01-05']
data.head()

,install_date,user_id,channel_group,city_level,age,gender,edu,finish_test,enter_wechat_group
0,2021-01-13,2.3746e+15,OV,新一线,24-30,male,Unknown,1,0
1,2021-01-06,1.88202e+15,Google-FB,六线及以下,24-30,female,Unknown,1,1
2,2021-01-14,3.7292e+15,华为小米,五线,31-40,female,undergraduate,0,0
3,2021-01-19,1.88278e+15,华为小米,三线,31-40,male,high,0,0
5,2021-01-09,8.26487e+14,Google-FB,二线,31-40,female,Unknown,0,1


In [34]:
len(data)

178271

In [35]:
data.groupby(["finish_test", "enter_wechat_group"]).size()

finish_test  enter_wechat_group
0            0                     111823
             1                       1476
1            0                      48676
             1                      16296
dtype: int64

##### Perpensity Score

In [37]:
# 是否完成定级测试 ～ 渠道 + 城市 + 教育

dummy = pd.get_dummies(data.iloc[:, 2:6])
new_data = pd.concat([data, dummy], axis = 1)
new_data

,install_date,user_id,channel_group,city_level,age,gender,edu,finish_test,enter_wechat_group,channel_group_Apple Store,...,age_-18,age_18-23,age_24-30,age_31-40,age_41-50,age_50-,age_Unknown,gender_Unknown,gender_female,gender_male
0,2021-01-13,2.3746e+15,OV,新一线,24-30,male,Unknown,1,0,0,...,0,0,1,0,0,0,0,0,0,1
1,2021-01-06,1.88202e+15,Google-FB,六线及以下,24-30,female,Unknown,1,1,0,...,0,0,1,0,0,0,0,0,1,0
2,2021-01-14,3.7292e+15,华为小米,五线,31-40,female,undergraduate,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,2021-01-19,1.88278e+15,华为小米,三线,31-40,male,high,0,0,0,...,0,0,0,1,0,0,0,0,0,1
5,2021-01-09,8.26487e+14,Google-FB,二线,31-40,female,Unknown,0,1,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511375,2021-01-16,1.12631e+15,华为小米,二线,31-40,male,high,0,0,0,...,0,0,0,1,0,0,0,0,0,1
511376,2021-01-16,2.21703e+15,Apple Store,新一线,31-40,female,undergraduate,0,0,1,...,0,0,0,1,0,0,0,0,1,0
511384,2021-01-10,2.90237e+15,OV,二线,31-40,female,undergraduate,0,0,0,...,0,0,0,1,0,0,0,0,1,0
511385,2021-01-10,4.09863e+15,Google-FB,Unknown,50-,male,Unknown,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [38]:
x = new_data.iloc[:, 9:35]
y = new_data[['finish_test']]

model = LogisticRegression(solver='liblinear', random_state=0).fit(x, y)


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [39]:
model.score(x,y)

0.6546998670563355